In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5
# Add modules to the path
import sys
import os
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git\\freely_moving_photometry_analysis')
import peakutils
from matplotlib import colors, cm
from scipy.signal import decimate
import copy
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import pickle
from utils.plotting import HeatMapParams
from utils.plotting import heat_map_and_mean
from utils.plotting import get_photometry_around_event
from scipy import stats
from utils.large_reward_omission_utils import plot_mean_trace_for_condition

In C:\Users\francescag\AppData\Local\Continuum\miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\francescag\AppData\Local\Continuum\miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\francescag\AppData\Local\Continuum\miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [21]:
root_directory = 'W:\photometry_2AC\processed_data'
figure_dir = 'W:\\photometry_2AC\\plots\\SNL_photo21_26\\omission_large_reward_experiment'
processed_data_dir = os.path.join(root_directory, 'large_rewards_omissions_data')
block_data_file = os.path.join(processed_data_dir, 'block_data_all_mice.csv')
all_reward_block_data = pd.read_pickle(block_data_file)

In [22]:
all_reward_block_data['time points'].reset_index(drop=True)[0]

array([-8.    , -7.9999, -7.9998, ...,  7.9998,  7.9999,  8.    ])

In [ ]:
font = {'size'   : 12}

matplotlib.rc('font', **font)

mouse_name = 'SNL_photo30'
contra_trials = all_reward_block_data[(all_reward_block_data['mouse'] == mouse_name) & (all_reward_block_data['side'] == 'contra')]
ipsi_trials = all_reward_block_data[(all_reward_block_data['mouse'] == mouse_name) & (all_reward_block_data['side'] == 'ipsi')]
timepoints = all_reward_block_data['time points'].reset_index(drop=True)[0]
fig, ax = plt.subplots(2,1) #, figsize=(10,16))
plot_mean_trace_for_condition(ax[0], contra_trials, timepoints,
                              'reward contra', error_bar_method='sem', save_location=processed_data_dir)
lg1 = ax[0].legend(title='',bbox_to_anchor=(1., .8, .15,.2), loc='upper left')
plot_mean_trace_for_condition(ax[1],ipsi_trials, timepoints,
                              'reward ipsi', error_bar_method = 'sem', save_location=processed_data_dir)
lg2 = ax[1].legend(title='',bbox_to_anchor=(1., .8, .15,.2), loc='upper left')

plt.tight_layout()

plt.savefig(os.path.join(figure_dir, 'example_mouse{}.pdf'.format(mouse_name)))


In [10]:
font = {'size'   : 12}

matplotlib.rc('font', **font)

mouse_name = 'SNL_photo26'
all_trials = all_reward_block_data[(all_reward_block_data['mouse'] == mouse_name)]
timepoints = all_reward_block_data['time points'].reset_index(drop=True)[0]
fig, ax = plt.subplots(1,1) #, figsize=(10,16))
plot_mean_trace_for_condition(ax, all_trials, timepoints,
                              'reward', error_bar_method='sem', save_location=processed_data_dir)
lg1 = ax.legend(title='',bbox_to_anchor=(1., .8, .15,.2), loc='upper left')
ax.set_ylim([-1.5, 4.1])
plt.tight_layout()

plt.savefig(os.path.join(figure_dir, 'example_mouse{}_both_sides.pdf'.format(mouse_name)))


In [23]:
def francescas_plot(ax, data, show_err_bar=False, mean_linewidth=4, mean_line_color='blue', legend=False):
    """Plot lines for all mice from early to late.
    """
    data.plot(ax=ax, color='gray', legend=False)
    data.mean(1).plot(ax=ax,linewidth=mean_linewidth,color=mean_line_color)

    if show_err_bar:
        yerr = data.std(axis=1)

        plt.errorbar(np.array([0,1]), data.mean(1), yerr, color=mean_line_color, linewidth=4)

    # drop the left and bottom spine for cool looking effect
    ax.spines['left'].set_position(('outward', 10))
    ax.spines['bottom'].set_position(('outward', 10))
    # Hide the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)


In [24]:
# find mean traces and downsample
avg_traces = all_reward_block_data.groupby(['mouse', 'reward'])['traces'].apply(np.mean)
decimated = [decimate(trace[int(len(trace)/2):], 10) for trace in avg_traces]
avg_traces = avg_traces.reset_index()
avg_traces['decimated'] = pd.Series([_ for _ in decimated])

In [25]:
# find peak size in pre condition and look for same time point in action
# alternatively, just find first peak for both. let's try that first. 

#plt.plot(avg_traces['decimated'].iloc[0])

first_peak_ids = [peakutils.indexes(i)[0] for i in avg_traces['decimated']]
avg_traces['peakidx'] = first_peak_ids
peaks = [np.mean(trace[:600]) for idx, trace in zip(first_peak_ids, avg_traces['decimated'])]
avg_traces['peak'] = peaks
avg_traces.set_index(['mouse', 'reward'])

normal_peak = avg_traces[avg_traces['reward']=='normal']['peak']
large_reward_peak = avg_traces[avg_traces['reward']=='large reward']['peak']
omission_peak = avg_traces[avg_traces['reward']=='omission']['peak']
stat1, pval1 = stats.ttest_rel(normal_peak, large_reward_peak)
stat2, pval2 = stats.ttest_rel(normal_peak, omission_peak)

In [26]:
avg_traces.head()

,mouse,reward,traces,decimated,peakidx,peak
0,SNL_photo22,large reward,"[0.11859116097908513, 0.1185452072411518, 0.11...","[0.001321999956787953, -0.0007689035317111923,...",244,-0.225620
1,SNL_photo22,normal,"[0.06827525251338239, 0.0682808154930902, 0.06...","[0.0896267660711066, 0.08744151377900143, 0.08...",1038,-0.238696
2,SNL_photo22,omission,"[0.011178706976070037, 0.011235044676938862, 0...","[0.17792240070082369, 0.1761558710423434, 0.17...",1015,-0.323676
3,SNL_photo26,large reward,"[0.05037895088930147, 0.050476736225644914, 0....","[7.327177380939465e-05, -0.00350988269789041, ...",352,-0.150822
4,SNL_photo26,normal,"[0.08096870900435212, 0.08093620336087026, 0.0...","[-0.050552494226527314, -0.052069352951295736,...",1906,-0.278525


In [27]:
# We run a repeated measures anova to check for a main effect of reward.
# Subsequently, we want to do pairwise testing between the three reward conditions. Need to correct for multiple comparisons
from statsmodels.stats.anova import AnovaRM
from statsmodels.stats.multitest import multipletests


In [28]:
#help(multipletests)
reject, corrected_pvals, corrected_alpha_sidak, corrected_bonf = multipletests([pval1, pval2], method='bonferroni')

print(corrected_pvals)

[0.87007311 1.        ]


In [29]:
df1 = avg_traces
df_for_plot = df1.pivot(index='reward', columns='mouse', values='peak').sort_values('reward', ascending=False)

In [30]:
df_for_plot

mouse,SNL_photo22,SNL_photo26
reward,,
omission,-0.323676,-0.127278
normal,-0.238696,-0.278525
large reward,-0.225620,-0.150822


In [31]:
fig, ax = plt.subplots(figsize=[4,5])
francescas_plot(ax, df_for_plot, mean_line_color='#7FB5B5', mean_linewidth=4, show_err_bar=False)
plt.xticks([0, 1, 2], ['omission', 'normal (2ul)', '3 x reward'], fontsize=12)
plt.ylabel('Z-scored fluorescence', fontsize=12)
ax.set_xlabel(' ')
#ax.text(1.2, 3, 'p-value = {0:.6f}'.format(corrected_pvals[1]))
#ax.text(0.1, 3, 'p-value = {0:.6f}'.format(corrected_pvals[0]))

# show significance stars 
# for first comparison
y = df_for_plot.T['large reward'].max() + .2
h = .1
plt.plot([0, 0, 1, 1], [y, y+h, y+h, y],c='k',lw=1)
ax.text(.5, y+h, 'n.s.', ha='center', fontsize=12)
# for second comparison
l = .2
plt.plot([1, 1, 2, 2], [y+l, y+h+l, y+h+l, y+l],c='k', linewidth=1)
ax.text(1.5, y+h+l, 'n.s.', ha='center', fontsize=12)
ax.set_ylim([-1, 3.4])
plt.tight_layout()